In [1]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_rows', 500)

In [2]:
master = pd.read_excel('wingspan-card-list.xlsx', sheet_name='Master')
bonus = pd.read_excel('wingspan-card-list.xlsx', sheet_name='Bonus cards')

In [3]:
# Isolate later expansions
oceania = master[master['Expansion'] == 'oceania']
oceania = oceania.sort_values(by=['Common name'], na_position='first')

# Index according to legacy rules 
master = master[(master['Expansion'].isin(['originalcore', 'core', 'swiftstart', 'european', 'chinesepromo']) | pd.isna(master['Expansion']))]
master = master.sort_values(by=['Common name'], na_position='first')
master = master.reset_index(drop=True)
master = master.drop([0, 1])

In [4]:
### Append later expansions
master = master.append(oceania)

In [5]:
master['id'] = master.index
master['Common name'] = master['Common name'].map(lambda s: s.strip())
bonus['id'] = bonus.index + 1000

In [6]:
master = master[master['Expansion'] != 'chinesepromo']

In [7]:
rulings = pd.read_csv('Wingspan - Rulings.tsv', sep='\t', header=None, names=['id', 'general', 'specific', 'text', 'source'])
rulings['text'] = rulings.text.map(lambda t: re.sub(r'\\textbf\{([\w \'\-\[\]]+)\}', r'<strong>\1</strong>', re.sub(r'\\textit\{([\w \'\-\[\]]+)\}', r'<i>\1</i>', t)))

general = rulings[~pd.isna(rulings['general'])].drop(['specific'], axis=1)
general['general'] = general.general.map(lambda t: re.sub(r'\$\\Rightarrow\$', '➔', t))
specific = rulings[~pd.isna(rulings['specific'])].drop(['general'], axis=1)
specific['specific'] = specific['specific'].map(lambda s: s.strip())
grouped = specific.groupby(by='specific').apply(lambda group: list(map(lambda t: {'text': t[0], 'source': t[1]}, zip(group['text'], group['source']))))

In [8]:
general.reset_index(drop=True, inplace=True)
general.drop(['id'], axis=1, inplace=True)
general.columns = ['name', 'text', 'source']
master['rulings'] = master['Common name'].map(lambda name: grouped[name] if name in grouped else [])
bonus['rulings'] = bonus['Name'].map(lambda name: grouped[name] if name in grouped else [])

In [9]:
master.sort_values(by='Common name', inplace=True)

In [10]:
master.to_json('generated/master.json', orient='records', index=True)
bonus.to_json('generated/bonus.json', orient='records', index=True)
general.to_json('generated/general.json', orient='index')